In [1]:
# warning: unused! deprecated!
import numpy as np
import pickle
import math
from scipy.stats import sem
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import statsmodels.formula.api as smf
import CMR_IA as cmr
import scipy as sp
sns.set_context('paper')
pd.set_option('display.max_columns', None)

In [2]:
with open("../Data/simu6b_design.pkl", 'rb') as inp:
    df_study = pickle.load(inp)
    df_test = pickle.load(inp)
# df_study = df_study.loc[df_study.session < 500]
# df_test = df_test.loc[df_test.session < 500]

In [3]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,session
0,153,736,BOTTLE,INDIAN,0
1,145,1198,BODY,ROBE,0
2,1391,1059,STRAWBERRY,PICKLE,0
3,140,838,BLUEBERRY,LOCK,0
4,1132,702,PROTON,HELMET,0
...,...,...,...,...,...
35995,176,1511,BROCCOLI,TROUT,999
35996,1463,1320,TICK,SKULL,999
35997,129,1261,BIRD,SEAGULL,999
35998,1271,96,SERVANT,BASKETBALL,999


In [4]:
df_test

,test_itemno,test_item,correct_ans,order,rep,test,session,test_pos,pair_idx
0,327,CLOTHES,210,2,1,1,0,1,11
1,736,INDIAN,153,2,5,1,0,2,6
2,140,BLUEBERRY,838,1,1,1,0,3,1
3,1391,STRAWBERRY,1059,1,1,1,0,4,5
4,819,LIGHTER,1251,1,5,1,0,5,8
...,...,...,...,...,...,...,...,...,...
23995,1545,VALLEY,1018,2,3,2,999,20,12000
23996,1511,TROUT,176,2,1,2,999,21,11992
23997,1632,YOKE,809,1,5,2,999,22,11991
23998,1393,STREET,482,1,5,2,999,23,11997


### Run CMR-IA

In [5]:
s_mat = np.load('../Data/wordpools/ltp_FR_similarity_matrix.npy')

In [6]:
params = cmr.make_default_params()
params.update(
    beta_enc = 0.5,
    beta_rec = 0.45,
    beta_cue = 0.45,
    beta_rec_post = 0.5,
    gamma_fc = 0.7,
    gamma_cf = 0.5,
    s_fc = 0.1,
    s_cf = 0.1,
    c_thresh = 0.01,
    c_thresh_ass = 0.01,
    a = 2800,
    b = 2.14,
    m = 0,
    n = 1,
    kappa = 0.08,
    lamb = 0.05,
    eta = 0.02,
    phi_s = 1,
    phi_d = 0.6,
    nitems_in_accumulator = 10,
    d_ass = 0.8
)
params

{'beta_enc': 0.5,
 'beta_rec': 0.45,
 'beta_rec_new': 0.5,
 'beta_rec_post': 0.5,
 'beta_distract': None,
 'phi_s': 1,
 'phi_d': 0.6,
 's_cf': 0.1,
 's_fc': 0.1,
 'kappa': 0.08,
 'eta': 0.02,
 'omega': 8,
 'alpha': 4,
 'c_thresh': 0.01,
 'c_thresh_ass': 0.01,
 'd_ass': 0.8,
 'lamb': 0.05,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 10,
 'max_recalls': 50,
 'learn_while_retrieving': False,
 'a': 2800,
 'b': 2.14,
 'm': 0,
 'n': 1,
 'c1': 0,
 'gamma_fc': 0.7,
 'gamma_cf': 0.5}

In [7]:
df_simu, f_in = cmr.run_norm_cr_multi_sess(params,df_study, df_test ,s_mat)
df_simu['test_pos'] = np.tile(np.arange(1,25),1000)
df_simu

CMR Time: 22.47266149520874


,session,test_itemno,s_resp,s_rt,test_pos
0,0,327,819.0,390.0,1
1,0,736,299.0,370.0,2
2,0,140,736.0,400.0,3
3,0,1391,153.0,420.0,4
4,0,819,1251.0,400.0,5
...,...,...,...,...,...
23995,999,1545,1393.0,470.0,20
23996,999,1511,482.0,370.0,21
23997,999,1632,1018.0,1010.0,22
23998,999,1393,1632.0,420.0,23


In [8]:
df_simu = df_simu.merge(df_test,on=['session','test_itemno','test_pos'])
df_simu['correct'] = df_simu.s_resp == df_simu.correct_ans
df_simu

,session,test_itemno,s_resp,s_rt,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct
0,0,327,819.0,390.0,1,CLOTHES,210,2,1,1,11,False
1,0,736,299.0,370.0,2,INDIAN,153,2,5,1,6,False
2,0,140,736.0,400.0,3,BLUEBERRY,838,1,1,1,1,False
3,0,1391,153.0,420.0,4,STRAWBERRY,1059,1,1,1,5,False
4,0,819,1251.0,400.0,5,LIGHTER,1251,1,5,1,8,True
...,...,...,...,...,...,...,...,...,...,...,...,...
23995,999,1545,1393.0,470.0,20,VALLEY,1018,2,3,2,12000,False
23996,999,1511,482.0,370.0,21,TROUT,176,2,1,2,11992,False
23997,999,1632,1018.0,1010.0,22,YOKE,809,1,5,2,11991,False
23998,999,1393,1632.0,420.0,23,STREET,482,1,5,2,11997,False


In [9]:
df_cond = df_simu.groupby(["pair_idx","test"])['order'].mean().to_frame(name='corr_rate').reset_index()
df_cond = df_cond.pivot_table(index='pair_idx',columns='test',values='corr_rate').reset_index()
df_cond.columns = ['pair_idx','test1','test2']
def cond(x):
    test1 = x['test1']
    test2 = x['test2']
    if test1 == 1 and test2 == 1:
        return 'F-F'
    elif test1 == 1 and test2 == 2:
        return 'F-B'
    elif test1 == 2 and test2 == 1:
        return 'B-F'
    elif test1 == 2 and test2 == 2:
        return 'B-B'

df_cond['cond'] = df_cond.apply(lambda x:cond(x),axis=1)
df_cond['cong'] = df_cond.apply(lambda x: 'Identical' if x['cond'] == 'F-F' or x['cond'] == 'B-B' else 'Reversed',axis=1)
df_cond

,pair_idx,test1,test2,cond,cong
0,1,1.0,2.0,F-B,Reversed
1,2,1.0,2.0,F-B,Reversed
2,3,1.0,2.0,F-B,Reversed
3,4,1.0,1.0,F-F,Identical
4,5,1.0,1.0,F-F,Identical
...,...,...,...,...,...
11995,11996,1.0,2.0,F-B,Reversed
11996,11997,1.0,1.0,F-F,Identical
11997,11998,2.0,2.0,B-B,Identical
11998,11999,2.0,1.0,B-F,Reversed


In [10]:
pairidx2cond = df_cond.loc[:,['pair_idx','cond']].set_index("pair_idx").to_dict()['cond']
pairidx2cong = df_cond.loc[:,['pair_idx','cong']].set_index("pair_idx").to_dict()['cong']
df_simu['cond'] = df_simu.apply(lambda x:pairidx2cond[x['pair_idx']],axis=1)
df_simu['cong'] = df_simu.apply(lambda x:pairidx2cong[x['pair_idx']],axis=1)
df_simu.head(24)


,session,test_itemno,s_resp,s_rt,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct,cond,cong
0,0,327,819.0,390.0,1,CLOTHES,210,2,1,1,11,False,B-B,Identical
1,0,736,299.0,370.0,2,INDIAN,153,2,5,1,6,False,B-B,Identical
2,0,140,736.0,400.0,3,BLUEBERRY,838,1,1,1,1,False,F-B,Reversed
3,0,1391,153.0,420.0,4,STRAWBERRY,1059,1,1,1,5,False,F-F,Identical
4,0,819,1251.0,400.0,5,LIGHTER,1251,1,5,1,8,True,F-F,Identical
5,0,299,819.0,330.0,6,CHILD,1282,1,5,1,4,False,F-F,Identical
6,0,856,299.0,350.0,7,MAID,607,2,3,1,10,False,B-F,Reversed
7,0,1132,1282.0,380.0,8,PROTON,702,1,5,1,3,False,F-B,Reversed
8,0,1372,1132.0,300.0,9,STAPLER,1169,2,3,1,12,False,B-F,Reversed
9,0,1198,702.0,390.0,10,ROBE,145,2,3,1,7,False,B-F,Reversed


### PLOT

In [11]:
dfgp_test_order = df_simu.groupby(['rep','test','order']).correct.mean().to_frame(name='corr_rate').reset_index()
dfgp_test_order

,rep,test,order,corr_rate
0,1,1,1,0.000000
1,1,1,2,0.000000
2,1,2,1,0.000000
3,1,2,2,0.000000
4,3,1,1,0.077993
5,3,1,2,0.085053
6,3,2,1,0.081918
7,3,2,2,0.087588
8,5,1,1,0.151227
9,5,1,2,0.141288


In [12]:
def contingency_tab(x):
    tmp_x = pd.pivot_table(x,index='pair_idx',columns='test', values= 'correct')
    return pd.crosstab(index=tmp_x[2],columns=tmp_x[1])

In [13]:
dfgp_rep_cong = df_simu.groupby(['rep','cong']).apply(contingency_tab)
dfgp_rep_cong

1                   0.0    1.0
rep cong      2               
1   Identical 0.0  1986    NaN
    Reversed  0.0  2014    NaN
3   Identical 0.0  1729  154.0
              1.0   146   10.0
    Reversed  0.0  1630  148.0
              1.0   169   14.0
5   Identical 0.0  1464  241.0
              1.0   222   48.0
    Reversed  0.0  1483  258.0
              1.0   246   38.0

In [14]:
df_simu.groupby(['rep','cong','test']).correct.mean()

rep  cong       test
1    Identical  1       0.000000
                2       0.000000
     Reversed   1       0.000000
                2       0.000000
3    Identical  1       0.080432
                2       0.076508
     Reversed   1       0.082611
                2       0.093320
5    Identical  1       0.146329
                2       0.136709
     Reversed   1       0.146173
                2       0.140247
Name: correct, dtype: float64

In [15]:
# pd.crosstab(index=a[2],columns=a[1])

In [16]:
# fig,ax = plt.subplots(figsize=(5,5))
# ax.spines[['right', 'top']].set_visible(False)
# ax.tick_params(axis="y",direction="in")
# ax.tick_params(axis="x",direction="in")
# sns.lineplot(data=df_laggp, x='lag', y='corr_rate', linewidth=2, marker='o', markersize = 7, hue='order',palette='tab10')
# plt.ylim([0,1])
# plt.xlabel('Test Lag')
# plt.ylabel('Probablility of Recall')
# L = plt.legend(title='Direction',loc='upper right')
# L.get_texts()[0].set_text('Forward')
# L.get_texts()[1].set_text('Backward')
# # plt.savefig('fig/simu6a_recall.pdf')
# plt.show()